In [ ]:
import pandas as pd
import numpy as np
from keras.models import load_model
from sklearn.externals import joblib


def get_data_dir():
    return '../data/'


def merged_file_path():
    return get_data_dir()+'Merged_Data_Excel_Test.xlsx'


def train_file_path():
    return get_data_dir()+'train_data.pkl'


def scaler_file_path():
    return get_data_dir()+'scaler.pkl'


def model_file_path():
    return get_data_dir()+'model_v1.h5'


df = pd.read_excel(merged_file_path(),index_col=0,parse_dates=True )
scaler = joblib.load(scaler_file_path())

scaled = scaler.transform(df)

print("scaled")

timesteps=600
rowcount = scaled.shape[0]
close_price_col_idx=3


X_list=[]
Y_list=[]


for i in range(rowcount-1,timesteps,-1):
    target_row=scaled[i]
    y_val = target_row[close_price_col_idx]
    start_row=(i-1)-(timesteps)
    end_row=i-1

    Y_list.append(y_val)
    x=scaled[start_row:end_row]
    X_list.append(x)


X=np.asarray(X_list)
Y_hat=np.asarray(Y_list)

print("Data prepared")

model = load_model(model_file_path())


In [ ]:
X.shape

In [ ]:
x1=X[1]

In [ ]:
x1.shape

In [ ]:
x1[0]

In [ ]:
x1=x1.reshape(1,600,17)

In [ ]:
x1[0][0]

In [ ]:
y1=model.predict(x1)

In [ ]:
y1

In [ ]:
Y_hat[0]

In [ ]:
x1_unscaled=scaler.inverse_transform(x1[0])

In [ ]:
x1_unscaled.shape

In [ ]:
x1_unscaled[0]

In [ ]:
close_prices=scaled[:,close_price_col_idx]

In [ ]:
close_prices.shape

In [ ]:
close_prices=close_prices[timesteps+1:]

In [ ]:
cp_df=pd.DataFrame(data=close_prices,columns=['close_price'])

In [ ]:
cp_df.head(5)

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import cufflinks as cf

init_notebook_mode(connected=True)
cf.go_offline()

In [ ]:
cp_df.iplot()

In [ ]:
len(cp_df)

In [ ]:
len(Y_hat)

In [ ]:
cp_df['y_hat']=Y_hat

In [ ]:
cp_df.head(5)

In [ ]:
cp_df.iplot()

In [ ]:
num_samples=X.shape[0]

In [ ]:
num_samples


In [ ]:
y_out=[]
for i in range(5):
    x=X[i]
    print(x.shape)
    print(x[0])
    x=x.reshape(1,x.shape[0],x.shape[1])
    y=model.predict(x)
    print(y)
    y_out.append(y[0])

In [ ]:
len(y_out)

In [ ]:
cp_df['y_out']=np.asarray(y_out)

In [ ]:
cp_df.iplot()

In [ ]:
y_out